In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from benchmarkrewriter.benchmark_parser import BenchmarkParser, WorkerBenchmarkParser, WorkerEncoding
import os

path_to_results = r'C:\Users\huda\Downloads\\'
benchmark_path = '/home/dhu/Downloads/benchmarks_with_workers'

In [ ]:
def worker_flexibility(benchmark : WorkerEncoding):
    n_assignments = 0
    m = benchmark.n_machines()
    o = benchmark.n_operations()
    durations = benchmark.durations()
    w = durations.shape[2]
    for i in range(len(durations)):
        for j in range(len(durations[i])):
            for k in range(len(durations[i][j])):
                if durations[i][j][k] > 0:
                    n_assignments += 1
    average_assignments = n_assignments / o
    return average_assignments / (m*w)


def get_flexibility_and_dv_worker(benchmark):
    all = 0
    unique = []
    machines_available = 0
    durations = benchmark.durations()
    for i in range(len(durations)):
        for j in range(len(durations[i])):
            for k in range(len(durations[i][j])):
                if durations[i][j][k] > 0:
                    if durations[i][j][k] not in unique:
                        unique.append(durations[i][j][k])
                    all += 1
            if any([x > 0 for x in durations[i][j]]):
                machines_available+=1
    return worker_flexibility(benchmark), len(unique) / all

def remap(name):
    if name.startswith('_'):
        name = name[1:]
    values = name.split('_')
    if values[0].startswith('Behnke'):
        return 'Behnke'+values[1]
    if values[0].startswith('Brandimarte'):
        return 'BrandimarteMk'+values[1]
    if values[0].startswith('Chambers'):
        return 'ChambersBarnes'+values[1]
    if values[0].startswith('HurinkS'):
        return 'HurinkSdata'+values[1]
    if values[0].startswith('HurinkE'):
        return 'HurinkEdata'+values[1]
    if values[0].startswith('HurinkR'):
        return 'HurinkRdata'+values[1]
    if values[0].startswith('HurinkV'):
        return 'HurinkVdata'+values[1]
    if values[0].startswith('DP'):
        return 'DPpaulli'+values[1]
    if values[0].startswith('Kacem'):
        return 'Kacem'+values[1]
    if values[0].startswith('Fattahi'):
        return 'Fattahi'+values[1]
    return name

def read_benchmarks_workers(path):
    result = dict()
    benchmarks = os.listdir(path)
    for benchmark in benchmarks:
        #parser = BenchmarkParser()
        parser = WorkerBenchmarkParser()
        data = parser.parse_benchmark(path + '/' + benchmark)
        #f, dv = get_flexibility_and_dv(data)
        f, dv = get_flexibility_and_dv_worker(data)
        instance_name = remap(benchmark[2:-12])
        result[instance_name] = {'n_operations': data.n_operations(), 'flexibility': f, 'duration_variety': dv, 'n_machines': data.n_machines()}

    return result

In [ ]:
benchmark_data = read_benchmarks_workers(benchmark_path)

In [ ]:
import json
results_categories = os.listdir(path_to_results)
results = dict()
for category in results_categories:
    results[category] = dict()
    instances = os.listdir(category)
    for instance in instances:
        with open(instance, 'r') as f:
            ga_data = json.loads(f.read())
            # TODO: gather best, average, ...
            if instance not in results[category]:
                results[category][instance] = ga_data


In [ ]:
# plot best and average results

# plot all best results and best average results for every instance, colored in the category that performed best
labels = results_categories.keys().tolist()

best_category = dict()
for instance in benchmark_data:
    best_category[category] = []
    
for instance in benchmark_data:
    best = float('inf')
    best_categories = []
    for category in results_categories:
        # TODO: decide what to do with instances with multiple best, also do same for average
        if results_categories[category][instance]['Result']['Fitness']['Makespan'] < best:
            best = results_categories[category][instance]['Result']['Fitness']['Makespan']
            best_categories = []
            best_categories.append(category)
        elif results_categories[category][instance]['Result']['Fitness']['Makespan'] == best:
            best_categories.append(category)
    for category in best_categories:
        best_category[category].append(instance)

for category in best_category:
    n = [x['n_operations'] for x in best_category[category]]
    m = [x['n_machines'] for x in best_category[category]]
    plt.scatter(n, m, label=category)
    plt.xlabel('# Operations')
    plt.ylabel('# Machines')
    #...
plt.show()